이전에 배운 것 처럼 x가 1개인 선형 회귀를 **단순 선형 회귀(Simple Linear Regression)**이라고 한다. <br>
다수의 x로 부터 y를 예측하는 것은 **다중 선형 회귀(Multivariable Linear Regression)**이라고 한다.

## <strong> 1. 데이터에 대한 이해 </strong>

3개의 퀴즈 점수로 부터 최종 점수를 예측하는 모델을 만들어 보자. <br>
이때 독립 변수 x는 3개 이므로 수식으로는 다음과 같다.

$H(x)=w_1x_1+w_2x_2+w_3x_3+b$

## <strong> 2. 파이토치로 구현하기 </strong>

도구를 임포트하고 랜덤 시드를 고정하자

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

훈련 데이터를 선언하자

In [4]:
x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

가중치 w와 b를 선언하자.

In [5]:
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

[참고] **torch.zeros() 메서드**는 주어진 size에 맞게 0으로 구성된 텐서를 만들어준다.

In [10]:
print(torch.zeros((2, 3)))  # size를 tuple형태로 전달해도 된다.
print(torch.zeros(2, 3))
print(torch.zeros(2))
print(torch.zeros(1, 2))    

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])
tensor([[0., 0.]])


경사 하강법을 1,000회 반복해보자.

In [15]:
# optimizer 설정
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)

nb_epochs = 1000
for epoch in range(nb_epochs+1):
  # H(x) 계산
  hypothesis = w1*x1_train + w2*x2_train + w3*x3_train + b

  # cost 계산
  cost = torch.mean((hypothesis-y_train)**2)

  # H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
        print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/1000 hypothesis: tensor([152.4296, 183.9308, 180.8563, 196.9709, 140.4533]) Cost: 1.079151
Epoch  100/1000 hypothesis: tensor([152.3981, 183.9547, 180.8479, 196.9634, 140.4856]) Cost: 1.038372
Epoch  200/1000 hypothesis: tensor([152.3664, 183.9766, 180.8384, 196.9547, 140.5159]) Cost: 0.999699
Epoch  300/1000 hypothesis: tensor([152.3356, 183.9980, 180.8292, 196.9462, 140.5455]) Cost: 0.963022
Epoch  400/1000 hypothesis: tensor([152.3056, 184.0187, 180.8203, 196.9378, 140.5743]) Cost: 0.928264
Epoch  500/1000 hypothesis: tensor([152.2764, 184.0389, 180.8116, 196.9297, 140.6024]) Cost: 0.895299
Epoch  600/1000 hypothesis: tensor([152.2480, 184.0585, 180.8031, 196.9217, 140.6297]) Cost: 0.864043
Epoch  700/1000 hypothesis: tensor([152.2204, 184.0776, 180.7949, 196.9139, 140.6563]) Cost: 0.834398
Epoch  800/1000 hypothesis: tensor([152.1936, 184.0962, 180.7869, 196.9063, 140.6823]) Cost: 0.806270
Epoch  900/1000 hypothesis: tensor([152.1675, 184.1143, 180.7791, 196.8989, 140.70

## <strong> 3. 벡터와 행렬 연산으로 바꾸기 </strong>

위 코드에서는 3개의 독립변수 x를 데이터로 사용했고 이때 하나씩 Tensor로 선언해주었다. <br>
하지만 x의 개수가 늘어나면 위와 같은 방식을 하기에는 힘들다. <br>

이를 위해 **행렬 곱셈(벡터의 내적)**을 사용할 수 있다.

* 행렬의 곱셈 과정에서 이루어진 벡터 연산을 **벡터의 내적(Dot Product)**이라고 한다.

$H(x)=w_1x_1+w_2x_2+w_3x_3+b$를 두 벡터의 내적으로 다음과 같이 표현할 수 있다.

$\begin{pmatrix}
x_1 \, x_2 \, x_3
\end{pmatrix}̇
⋅
\begin{pmatrix}
w_1 \\
w_2 \\
w_3
\end{pmatrix}
= (x_1w_1 + x_2w_2 + x_3w_3)
$

그러면 이전 데이터의 내적을 표현해보자. <br>

$\begin{pmatrix}
x_{11} \, x_{12} \, x_{13} \\
x_{21} \, x_{22} \, x_{23} \\
x_{31} \, x_{32} \, x_{33} \\
x_{41} \, x_{42} \, x_{43} \\
x_{51} \, x_{52} \, x_{53} \\
\end{pmatrix}
⋅
\begin{pmatrix}
w_1 \\
w_2 \\
w_3
\end{pmatrix}
$

위 내적을 수행해주고 bias를 더해주면 이전 식과 동일한 수식이 된다.

## <strong> 4. 행렬 연산을 고려하여 파이토치로 구현하기 </strong>

행렬 연산을 고려해 이전 코드를 수정해보자.

In [14]:
# 데이터 선언
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  80], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

# w, b 선언
w = torch.zeros([3, 1], requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 선언
optimizer = optim.SGD([w, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs+1):
  # H(x) 선언
  hypothesis = x_train.matmul(w)+b

  # cost 계산
  cost = torch.mean((hypothesis-y_train)**2)

  # H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([66.7178, 80.1701, 76.1025, 86.0194, 61.1565]) Cost: 9537.694336
Epoch    2/20 hypothesis: tensor([104.5421, 125.6208, 119.2478, 134.7862,  95.8280]) Cost: 3069.590088
Epoch    3/20 hypothesis: tensor([125.9858, 151.3882, 143.7087, 162.4333, 115.4844]) Cost: 990.670288
Epoch    4/20 hypothesis: tensor([138.1429, 165.9963, 157.5768, 178.1071, 126.6283]) Cost: 322.481873
Epoch    5/20 hypothesis: tensor([145.0350, 174.2780, 165.4395, 186.9928, 132.9461]) Cost: 107.717064
Epoch    6/20 hypothesis: tensor([148.9423, 178.9730, 169.8976, 192.0301, 136.5279]) Cost: 38.687496
Epoch    7/20 hypothesis: tensor([151.1574, 181.6346, 172.4254, 194.8856, 138.5585]) Cost: 16.499043
Epoch    8/20 hypothesis: tensor([152.4131, 183.1435, 173.8590, 196.5043, 139.7097]) Cost: 9.365656
Epoch    9/20 hypothesis: tensor([153.1250, 183.9988, 174.6723, 197.4217, 140.3625]) Cost: 7.071114
Epoch   10/20 hyp